In [10]:
# import libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [11]:
def compute_ttt_empirical(times):
    """
    Calculates TTT plot empirical distribution
    
    Args:
        times: time to target array 
        
    Returns:
        sorted_times: sorted times
        probabilities: cumulative probabilities
    """
    n = len(times)
    sorted_times = np.sort(times)
    probabilities = np.array([(i + 0.5) / n for i in range(n)])
    return sorted_times, probabilities

In [12]:
def plot_ttt_comparison(data_dict, instance, target, output_dir='plots'):
    """
    Plots TTT
    
    Args:
        data_dict: dict {metaheuristic_name: time_array}
        instance: instance name
        target: target value
        output_dir: directory to save results
    """
    plt.figure(figsize=(10, 7))
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
    markers = ['o', 's', '^', 'D', 'v']
    
    for idx, (algorithm, times) in enumerate(data_dict.items()):
        sorted_times, probs = compute_ttt_empirical(times)
        
        plt.plot(sorted_times, probs, 
                marker=markers[idx % len(markers)],
                markersize=4,
                linewidth=2,
                label=algorithm,
                color=colors[idx % len(colors)],
                alpha=0.7)
    
    plt.xlabel('Time to target (ms)', fontsize=12)
    plt.ylabel('Cumulative Probability', fontsize=12)
    plt.title(f'TTT Plot - Instance: {instance}, Target: {target}', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3, linestyle='--')
    plt.legend(loc='lower right', fontsize=11, framealpha=0.9)
    plt.ylim(0, 1)
    plt.xlim(left=0)
    
    # Save plot
    Path(output_dir).mkdir(exist_ok=True)
    filename = f"{output_dir}/ttt_{instance}_{target}.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"Saved plot: {filename}")
    plt.close()

In [13]:
# Define CSV files
csv_files = {
    'GRASP': 'resultados_grasp_scqbf.csv',
    'Tabu Search': 'resultados_ts_scqbf.csv',
    'Genetic Algorithm': 'resultados_ga_scqbf.csv'
}

# Read CSVs
dfs = {}
for algorithm, filepath in csv_files.items():
    try:
        df = pd.read_csv(filepath)
        dfs[algorithm] = df
        print(f"Loaded: {filepath} ({len(df)} rows)")
    except FileNotFoundError:
        print(f"File not found: {filepath}")

Loaded: resultados_grasp_scqbf.csv (300 rows)
Loaded: resultados_ts_scqbf.csv (300 rows)
Loaded: resultados_ga_scqbf.csv (300 rows)


In [14]:
# Identify pairs (instance, target)
all_pairs = set()
for df in dfs.values():
    df['instancia'] = df['instancia'].str.split('/').str[-1].str.replace('.txt', '', regex=False)
    pairs = df[['instancia', 'target']].drop_duplicates()
    all_pairs.update(pairs.itertuples(index=False, name=None))

all_pairs = sorted(all_pairs)
print(f"\nPairs (instance, target) found: {len(all_pairs)}")


Pairs (instance, target) found: 3


In [15]:
# Generate plot for each pair
for instance, target in all_pairs:
    print(f"\nPlotting: instance={instance}, target={target}")
    
    # Extract times
    data_dict = {}
    for algorithm, df in dfs.items():
        mask = (df['instancia'] == instance) & (df['target'] == target)
        times = df.loc[mask, 'tempo_para_target_ms'].values
        
        if len(times) > 0:
            data_dict[algorithm] = times
            print(f"  {algorithm}: {len(times)} runs")
        else:
            print(f"  {algorithm}: No data")
    
    # Plot
    if len(data_dict) > 0:
        plot_ttt_comparison(data_dict, instance, target)
    else:
        print(f" No metaheuristic found. Skipping...")


Plotting: instance=n200p3, target=4715.38
  GRASP: 100 runs
  Tabu Search: 100 runs
  Genetic Algorithm: 100 runs
Saved plot: plots/ttt_n200p3_4715.38.png

Plotting: instance=n400p2, target=6248.32
  GRASP: 100 runs
  Tabu Search: 100 runs
  Genetic Algorithm: 100 runs
Saved plot: plots/ttt_n400p2_6248.32.png

Plotting: instance=n400p2, target=10486.78
  GRASP: 100 runs
  Tabu Search: 100 runs
  Genetic Algorithm: 100 runs
Saved plot: plots/ttt_n400p2_10486.78.png
